In [ ]:
import matplotlib.pyplot as plt
import pandas as pd 
import datetime as dt
import pickle

In [ ]:
import pandas as pd
df = pd.read_csv('reddit_posts_2022_07_21-10_16_58_AM_no_duplicates_personnal_corrected.csv', index_col=0)
df.info()

### Data Preprocessing

In [ ]:
import re, string, unicodedata
import nltk
import contractions # expanding contractions
import inflect # natural language related tasks of generating plurals, singular nouns, etc.
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from nltk.tokenize import TweetTokenizer
import string


#nltk.download('punkt')
#nltk.download('stopwords')
#nltk.download('wordnet')
#nltk.download('averaged_perceptron_tagger')
#nltk.download('omw-1.4')

In [ ]:
import sys
sys.path.append("../utils/preprocess")

In [ ]:
from preprocess import Preprocess
prep = Preprocess()

In [ ]:
#functions for replacements in text

def multiple_replacer(*key_values):
    replace_dict = dict(key_values)
    replacement_function = lambda match: replace_dict[match.group(0)]
    pattern = re.compile("|".join([re.escape(k) for k, v in key_values]), re.M)
    return lambda string: pattern.sub(replacement_function, string)

def multiple_replace(string, *key_values):
    return multiple_replacer(*key_values)(string)


In [ ]:
def preprocess_post(post, replacements, stemming=True, numbers_processing='replace'):
    
    post = prep.replace_contractions(post)
    post = prep.replace_special_words(post)
    post = prep.replace_hashtags_URL_USER(post, mode_URL="delete", mode_Mentions="delete", mode_Hashtag="replace")
    post = prep.tokenize(post)
    post = prep.remove_punctuation(post)
    post = prep.preprocess_emojis(post)
    post = prep.preprocess_emoticons(post)
    post = prep.remove_non_ascii(post)
    post = prep.to_lowercase(post)
    post = prep.replace_numbers(post, mode=numbers_processing)
    post = prep.lemmatize_verbs(post)
    post = prep.remove_stopwords(post)
    if replacements:
        post = " ".join(post)
        for i in range(len(replacements)):
            post= multiple_replace(post, *replacements[i])
        post=post.split()
    if stemming:
        post = prep.stem_words(post)
    return post

def preprocess_posts(posts, *replacements, stemming=True, numbers_processing='replace'):
    return [" ".join(preprocess_post(post, replacements, stemming, numbers_processing)) for post in posts if post]

In [ ]:
replacements = [(u"view poll", u""), (u"httpurl", u"")]

df['long_covid_related_text_unstemmed_without_numbers'] = preprocess_posts(df['concatenated_sentences'], replacements, stemming=False, numbers_processing='delete')

In [ ]:
df.dropna(axis=0, how='any', thresh=None, subset=['long_covid_related_text_unstemmed_without_numbers'], inplace=True)

In [ ]:
indices = df[df['long_covid_related_text_unstemmed_without_numbers'].str.len() < 4 ]['long_covid_related_text_unstemmed_without_numbers'].index
df.drop(indices, inplace=True)

In [ ]:
df.to_csv('reddit_posts_2022_07_21-10_16_58_AM_no_duplicates_personnal_concatenated_clean.csv')

In [ ]:
df.info()

In [ ]:
# Use list comprehension to convert a list of lists to a flat list 
all_words = [item for elem in data_words for item in elem]
titled_all_words = [item for elem in titled_data_words for item in elem]

In [ ]:
from collections import Counter
word_frequencies = Couner(all_words)
word_frequencies.most_common(20)

### Co-occurences 

In [ ]:
from nltk import bigrams
import itertools
import collections

# Create list of lists containing bigrams in redit posts
terms_bigram = [list(bigrams(post)) for post in data_words]
# Flatten list of bigrams in clen tweets
bigrams = list(itertools.chain(*terms_bigram))
bigram_counts = collections.Counter(bigrams)

In [ ]:
NB_PAIRS = 100
bigram_counts.most_common(NB_PAIRS)